In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
dataset_dir = '../Data/label.txt'
org = np.loadtxt(dataset_dir)
# org = pd.read_csv(dataset_dir, sep=' ',dtype={'balance':np.float64}, header=None)

In [1]:
import platform 
platform.architecture()

('64bit', 'ELF')

In [6]:
org.shape

(371751, 60)

In [7]:
test_set = org[-10:, :]

In [8]:
misscolidx = 6
test_set_pre = np.delete(test_set, misscolidx, axis=1)
# org_train = np.delete(org[:10000], misscolidx, axis=1)

In [18]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import random
def RandomForest(x,y,rf_iteration):
    rfr = RandomForestRegressor(random_state=0, n_estimators=rf_iteration,  n_jobs=-1)
    rfr.fit(x,y)
    del x,y
    return rfr

def LightGBM(X, y, gbm_iteration):
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'learning_rate': 0.015,
        'cat_smooth': 10,
        'feature_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': 0
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=gbm_iteration,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=70)
    return gbm

rand_num = 10000
rf_iteration = 1000
gbm_iteration = 1000
rand_t = random.sample(range(org.shape[0]-10), rand_num)
x = np.delete(org[rand_t],misscolidx,axis=1)
y = org[rand_t,misscolidx]

import time
ts = time.clock()
m1 = RandomForest(x,y,rf_iteration)
te = time.clock()
print("Train RandomForest done. Time:{}s".format(te-ts))

ts = time.clock()
m2 = LightGBM(x,y, gbm_iteration)
te = time.clock()
print("Train LightGBM done. Time:{}s".format(te-ts))


res1 = m1.predict(test_set_pre)
print("****** RF result ******")
print("****** data_set={}, iterations={}".format(rand_num,rf_iteration))
print(res1)
print("****** L2 dist ******")
print(np.linalg.norm(res1 - org[-10:,misscolidx]))

res2 = m2.predict(test_set_pre)
print("****** LightGBM result ******")
print("****** data_set={}, iterations={}, best_iteration={}".format(rand_num, gbm_iteration, m2.best_iteration))
print(res2)
print("****** L2 dist ******")
print(np.linalg.norm(res2 - org[-10:,misscolidx]))

Train RandomForest done. Time:184.8327589999999s
[1]	valid_0's rmse: 3089.12
Training until validation scores don't improve for 70 rounds.
[2]	valid_0's rmse: 3059.27
[3]	valid_0's rmse: 3029.72
[4]	valid_0's rmse: 3000.34
[5]	valid_0's rmse: 2971.35
[6]	valid_0's rmse: 2942.43
[7]	valid_0's rmse: 2913.74
[8]	valid_0's rmse: 2885.26
[9]	valid_0's rmse: 2857.02
[10]	valid_0's rmse: 2829
[11]	valid_0's rmse: 2801.21
[12]	valid_0's rmse: 2773.48
[13]	valid_0's rmse: 2745.95
[14]	valid_0's rmse: 2718.57
[15]	valid_0's rmse: 2691.51
[16]	valid_0's rmse: 2664.67
[17]	valid_0's rmse: 2637.97
[18]	valid_0's rmse: 2611.45
[19]	valid_0's rmse: 2584.98
[20]	valid_0's rmse: 2558.83
[21]	valid_0's rmse: 2532.88
[22]	valid_0's rmse: 2507.21
[23]	valid_0's rmse: 2481.66
[24]	valid_0's rmse: 2456.31
[25]	valid_0's rmse: 2431.1
[26]	valid_0's rmse: 2406.01
[27]	valid_0's rmse: 2381.04
[28]	valid_0's rmse: 2356.43
[29]	valid_0's rmse: 2331.8
[30]	valid_0's rmse: 2307.62
[31]	valid_0's rmse: 2283.37
[32]

[295]	valid_0's rmse: 119.867
[296]	valid_0's rmse: 119.284
[297]	valid_0's rmse: 118.747
[298]	valid_0's rmse: 118.2
[299]	valid_0's rmse: 117.662
[300]	valid_0's rmse: 117.142
[301]	valid_0's rmse: 116.656
[302]	valid_0's rmse: 116.131
[303]	valid_0's rmse: 115.65
[304]	valid_0's rmse: 115.159
[305]	valid_0's rmse: 114.723
[306]	valid_0's rmse: 114.279
[307]	valid_0's rmse: 113.843
[308]	valid_0's rmse: 113.384
[309]	valid_0's rmse: 112.978
[310]	valid_0's rmse: 112.553
[311]	valid_0's rmse: 112.135
[312]	valid_0's rmse: 111.652
[313]	valid_0's rmse: 111.226
[314]	valid_0's rmse: 110.797
[315]	valid_0's rmse: 110.345
[316]	valid_0's rmse: 109.907
[317]	valid_0's rmse: 109.555
[318]	valid_0's rmse: 109.179
[319]	valid_0's rmse: 108.814
[320]	valid_0's rmse: 108.443
[321]	valid_0's rmse: 108.086
[322]	valid_0's rmse: 107.687
[323]	valid_0's rmse: 107.343
[324]	valid_0's rmse: 106.975
[325]	valid_0's rmse: 106.632
[326]	valid_0's rmse: 106.268
[327]	valid_0's rmse: 105.915
[328]	valid_0

[572]	valid_0's rmse: 73.953
[573]	valid_0's rmse: 73.9228
[574]	valid_0's rmse: 73.8623
[575]	valid_0's rmse: 73.776
[576]	valid_0's rmse: 73.7251
[577]	valid_0's rmse: 73.6571
[578]	valid_0's rmse: 73.5975
[579]	valid_0's rmse: 73.5314
[580]	valid_0's rmse: 73.4273
[581]	valid_0's rmse: 73.3714
[582]	valid_0's rmse: 73.3106
[583]	valid_0's rmse: 73.2732
[584]	valid_0's rmse: 73.2322
[585]	valid_0's rmse: 73.1526
[586]	valid_0's rmse: 73.0993
[587]	valid_0's rmse: 73.0423
[588]	valid_0's rmse: 73.006
[589]	valid_0's rmse: 72.9583
[590]	valid_0's rmse: 72.8973
[591]	valid_0's rmse: 72.8329
[592]	valid_0's rmse: 72.7436
[593]	valid_0's rmse: 72.6626
[594]	valid_0's rmse: 72.5876
[595]	valid_0's rmse: 72.5395
[596]	valid_0's rmse: 72.4737
[597]	valid_0's rmse: 72.4536
[598]	valid_0's rmse: 72.3887
[599]	valid_0's rmse: 72.3421
[600]	valid_0's rmse: 72.296
[601]	valid_0's rmse: 72.2577
[602]	valid_0's rmse: 72.1676
[603]	valid_0's rmse: 72.1165
[604]	valid_0's rmse: 72.0753
[605]	valid_0'

[855]	valid_0's rmse: 64.1439
[856]	valid_0's rmse: 64.1276
[857]	valid_0's rmse: 64.0795
[858]	valid_0's rmse: 64.0593
[859]	valid_0's rmse: 64.0336
[860]	valid_0's rmse: 64.021
[861]	valid_0's rmse: 64.0015
[862]	valid_0's rmse: 63.9862
[863]	valid_0's rmse: 63.9572
[864]	valid_0's rmse: 63.948
[865]	valid_0's rmse: 63.9275
[866]	valid_0's rmse: 63.9113
[867]	valid_0's rmse: 63.8842
[868]	valid_0's rmse: 63.8745
[869]	valid_0's rmse: 63.8385
[870]	valid_0's rmse: 63.8113
[871]	valid_0's rmse: 63.7828
[872]	valid_0's rmse: 63.7586
[873]	valid_0's rmse: 63.7225
[874]	valid_0's rmse: 63.7121
[875]	valid_0's rmse: 63.6943
[876]	valid_0's rmse: 63.674
[877]	valid_0's rmse: 63.6484
[878]	valid_0's rmse: 63.6256
[879]	valid_0's rmse: 63.6127
[880]	valid_0's rmse: 63.5978
[881]	valid_0's rmse: 63.5535
[882]	valid_0's rmse: 63.5512
[883]	valid_0's rmse: 63.5343
[884]	valid_0's rmse: 63.4969
[885]	valid_0's rmse: 63.4705
[886]	valid_0's rmse: 63.4371
[887]	valid_0's rmse: 63.4249
[888]	valid_0

In [6]:
print test_set_pre.shape
pre = m.predict(test_set_pre)
print pre
print org[-10:,misscolidx]

(10, 59)
[3016.07865625 2995.10821875 2981.98121875 2969.00540625 2960.70628125
 2949.569875   2945.0098125  2944.11915625 2947.92053125 4252.42178125]
[2983.75    2974.84375 2965.9375  2957.03125 2948.125   2939.21875
 2930.3125  2921.40625 2912.5     2903.59375]


In [11]:
np.linalg.norm(res1 - org[-10:,misscolidx]) 

822.9341630404211

In [7]:
res2 = pre #5000 3000
np.linalg.norm(res2 - org[-10:,misscolidx]) 

205.69489300545845

In [7]:
res3 = pre
np.linalg.norm(res3 - org[-10:,misscolidx]) 

1350.350164027599